###  Spam Email Classifier using Support Vector Machine Classifier

In [27]:
from sklearn import svm
from sklearn.metrics import accuracy_score

In [36]:
TRAIN_DIR = "train-mails"
TEST_DIR = "test-mails"
def make_dictionary(root_dir):
    all_words = []
    emails = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]
    for mail in emails:
        with open(mail) as m:
            for line in m:
                words = line.split()
                all_words += words
    dictionary = Counter(all_words)
    #dict_copy = deepcopy(dictionary)
    list_to_remove = list(dictionary)
    
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(2000)
    return dictionary

def extract_features(mail_dir):
    files = [os.path.join(mail_dir, f) for f in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), 2000))
    train_labels = np.zeros(len(files))
    count = 0
    doc_id = 0
    for f in files:
        with open(f) as fi:
            for i, line in enumerate(fi):
                if(i == 2):
                    words = line.split()
                    for word in words:
                        word_id = 0
                        for i, d in enumerate(dictionary):
                            if d[0] == word:
                                word_id = i
                                features_matrix[doc_id, word_id] = words.count(word)
            train_labels[doc_id] = 0
            filePathTokens = f.split('/')
            lastToken = filePathTokens[len(filePathTokens) - 1]
            if lastToken.__contains__("spmsg"):
                train_labels[doc_id] = 1
                count = count + 1
            doc_id = doc_id + 1
    return features_matrix, train_labels

In [37]:
dictionary = make_dictionary(TRAIN_DIR)
print('Reading and Processing Emails from File')

Reading and Processing Emails from File


In [ ]:
# for now, decrease the training time, by decraesing training data
# features_matrix = features_matrix(:len(features_matrix)/10)
# labels = labels(:len(labels)/10)

In [38]:
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

In [68]:
# Default Params for SVC -  kernel=Linear, c = 1, gamma = 1
model = svm.SVC()
print('Training model')
model.fit(features_matrix, labels)
predicted_labels = model.predict(test_features_matrix)
print('Accuracy Score')
accuracy_score(test_labels, predicted_labels)

Training model


C:\Users\Prashant\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.


Accuracy Score


0.8576923076923076

## Parameter Tuning

In [60]:
model = svm.SVC(kernel='rbf', C=100) #vary C = 1, 10, 100, 1000, 10000
model.fit(features_matrix, labels)
predicted_labels = model.predict(test_features_matrix)
print('Accuracy Score with RBF and C tuning')
accuracy_score(test_labels, predicted_labels)

C:\Users\Prashant\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.


Accuracy Score with RBF and C tuning


0.9769230769230769

In [67]:
model = svm.SVC(kernel='rbf', C = 100, gamma = 0.001)  #vary gamma = 10, 1, 0.1, 0.01, 0.001, 0.0001
model.fit(features_matrix, labels)
predicted_labels = model.predict(test_features_matrix)
print('Accuracy Score with C and gamma tuning')
accuracy_score(test_labels, predicted_labels)

Accuracy Score with C and gamma tuning


0.9769230769230769

#### This is the optimal Accuracy achieved with rbf kernel and C and gamma parameter tuning in SVC